# Day 36

Today I played around with the dataset I pulled in yesterday and joined it against the data I pulled from nflverse containing fantasy points scored per player. I wanted to see if players that are drafted in the lower rounds actually finished where they were drafted. In other words, did a player who was drafted in the first round of a 10-team standard league actually finish in the top 10 in terms of total fantasy points for the season?

In [2]:
import pandas as pd
import numpy as np
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

## Explore

In [3]:
query = """
SELECT *
FROM adp
"""

adp = pd.read_sql(query, conn)

In [4]:
pd.crosstab(adp.season, adp.scoring)

scoring,half-ppr,ppr,standard
season,,,
2008,0,0,360
2009,0,0,404
2010,0,428,420
2011,0,376,366
2012,0,186,290
2013,0,382,388
2014,0,388,326
2015,0,402,388
2016,0,376,336


The dataset contains mostly ppr and standard drafts. Standard drafts range from 2008 - 2021, ppr drafts range from 2010 to 2021.

## Analysis

Which players drafted in the first round actually met expectations? If so, how many seasons?

In [6]:
query = """
WITH season_summary AS (
    SELECT
        player_id,
        player_display_name AS name,
        season,
        position,
        recent_team AS team,
        ROUND(SUM(fantasy_points),2) AS tot_pts,
        ROUND(SUM(fantasy_points_ppr),2) AS tot_pts_ppr
    FROM weekly
    WHERE season_type = 'REG' 
        AND player_display_name IS NOT NULL
    GROUP BY player_id, player_display_name, season),
rankings AS (
    SELECT
        *,
        RANK() OVER(PARTITION BY season ORDER BY tot_pts DESC) AS r_pts,
        RANK() OVER(PARTITION BY season ORDER BY tot_pts_ppr DESC) AS r_pts_ppr
    FROM season_summary), 
joined AS (   
    SELECT * 
    FROM adp
    LEFT JOIN rankings
        ON rankings.name = adp.name
        AND rankings.season = adp.season
        AND rankings.position = adp.position
        -- AND rankings.team = adp.team -- this can cause missing values
    ), 
standard_top_performers AS (
    SELECT
        season,
        teams,
        scoring,
        name,
        position,
        team,
        adp_formatted,
        round,
        pick,
        tot_pts,
        -- tot_pts_ppr,
        r_pts
        -- r_pts_ppr
    FROM joined
    -- Only want to look at standard
    WHERE scoring = 'standard'
        AND (
            (teams = 10
                AND round = 1
                AND r_pts BETWEEN 1 AND 10)
            OR (teams = 12
                AND (round = 1 AND r_pts BETWEEN 1 AND 10) 
                OR (ROUND = 2 AND pick IN (1, 2) AND r_pts BETWEEN 1 AND 12)))),
ppr_top_performers AS (
    SELECT
        season,
        teams,
        scoring,
        name,
        position,
        team,
        adp_formatted,
        round,
        pick,
        -- tot_pts,
        tot_pts_ppr,
        -- r_pts
        r_pts_ppr
    FROM joined
    -- Only want to look at ppr
    WHERE scoring = 'ppr'
        AND (
            (teams = 10
                AND round = 1
                AND r_pts_ppr BETWEEN 1 AND 10)
            OR (teams = 12
                AND (round = 1 AND r_pts BETWEEN 1 AND 10) 
                OR (ROUND = 2 AND pick IN (1, 2) AND r_pts_ppr BETWEEN 1 AND 12)))
)
SELECT scoring, teams, name, position, COUNT(*) AS num_top_finishes
FROM ppr_top_performers
GROUP BY teams, name
HAVING num_top_finishes > 1
ORDER BY teams, num_top_finishes DESC
"""

pd.read_sql(query, conn)

,scoring,teams,name,position,num_top_finishes
0,ppr,10,Antonio Brown,WR,3
1,ppr,10,LeSean McCoy,RB,2
2,ppr,10,Ezekiel Elliott,RB,2
3,ppr,10,Davante Adams,WR,2
4,ppr,10,Calvin Johnson,WR,2
5,ppr,10,Alvin Kamara,RB,2
6,ppr,10,Aaron Rodgers,QB,2
7,ppr,12,Aaron Rodgers,QB,3
8,ppr,12,LeSean McCoy,RB,2
9,ppr,12,Christian McCaffrey,RB,2


Here I decided to look at just PPR and split it between 10 and 12-team leagues because with the higher league size there is a little bit of wiggle room since in a 12-team league a player must finish in the top 12 to be part of this table. You can see how Aaron Rodgers has 3 top finishes in 12-team but only 2 in 10-team leagues.  

LeSean McCoy and Aaron Rodgers are the only two players to appear in this table twice. Tomorrow I'll investigate why Antonio Brown is not in the 12-team section even though he has 3 top finishes in 10-team leagues. 